# 1、InMemoryChatMessageHistory的使用

场景1：记忆存储

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import PromptTemplate

# 1、ChatMessageHistory的实例化
history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

# 3、打印存储的消息
print(history.messages)

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


In [2]:
# 获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1 + 2 * 3 = ？")

response = llm.invoke(history.messages)
print(response.content)

根据数学中的运算规则，乘法优先于加法。所以，1 + 2 * 3 应该先计算乘法部分：

2 * 3 = 6

然后将结果加到1上：

1 + 6 = 7

因此，1 + 2 * 3 的结果是 7。


# 2、InMemoryChatMessageHistory的使用

举例1：以字符串的方式返回存储的信息

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1、ConversationBufferMemory的实例化
memory = InMemoryChatMessageHistory()

# 2、存储相关的消息
memory.add_message(HumanMessage(content="你好，我叫小明"))
memory.add_message(AIMessage(content="很高兴认识你"))

memory.add_message(HumanMessage(content="帮我回答一下1+2*3=?"))
memory.add_message(AIMessage(content="7"))

# 3、获取存储的信息
print(memory.messages)

# 说明：返回的字典结构的key叫history

[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='帮我回答一下1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例2：以消息列表的方式返回存储的信息

In [9]:
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {} 

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 1. 获取历史对象 (传入任意 session_id)
history = get_session_history("test_user_1")

# 2. 添加消息
history.add_user_message("这是用户说的话：1+1等于几？")
history.add_ai_message("这是AI说的话：等于2")

# 3. 打印结果
print(history.messages)

[HumanMessage(content='这是用户说的话：1+1等于几？', additional_kwargs={}, response_metadata={}), AIMessage(content='这是AI说的话：等于2', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例3：结合大模型、提示词模板的使用（PromptTemplate）

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
import os
import dotenv
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 1. 创建模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2. Prompt (使用 MessagesPlaceholder 替代手动拼接 history)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以与人类对话。"),
    MessagesPlaceholder(variable_name="history"), # 这是一个占位符，专门放历史记录
    ("human", "{question}"),
])

# 3. 定义 Chain (使用 LCEL 管道符 | 替代 LLMChain)
# 逻辑流：Prompt -> 模型 -> 输出转字符串
chain = prompt | llm | StrOutputParser()

# 4. 记忆管理 (替代 ConversationBufferMemory)
# 用一个简单的字典模拟数据库
user_1_history = InMemoryChatMessageHistory()
user_1_history.add_message(HumanMessage(content="小明喜欢什么？"))
user_1_history.add_message(AIMessage(content="小明喜欢编程。"))
store = {"user_1": user_1_history}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 使用 RunnableWithMessageHistory 包装 chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

# 5. 调用
response = chain_with_history.invoke(
    {"question": "你好，我的名字叫小明，你知道我喜欢什么吗？"},
    config={"configurable": {"session_id": "user_1"}}
)

print(response)

你好，小明！根据我们之前的对话，你提到你喜欢编程。不过，如果你还有其他特别的兴趣爱好或者喜欢的事物，也随时可以告诉我哦！


举例4：基于举例3，显式的设置meory的key的值


In [20]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1、创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2、提供提示词模板 (ChatPromptTemplate 替代 PromptTemplate)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以与人类对话。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# 3、提供Chain (使用管道符 | 构建)
chain = prompt | llm | StrOutputParser()

# 4、提供memory管理逻辑 (替代 ConversationBufferMemory)
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5、包装 Chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",      # 对应输入的 key
    history_messages_key="chat_history",  # 对应提示词模板中的占位符
)

# 6、调用
response_1 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小罗,我喜欢小鹿。"},
    config={"configurable": {"session_id": "user_1"}}
)

response_2 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？"},
    config={"configurable": {"session_id": "user_2"}}
)

print(store)
print(store["user_1"])
print(type(store["user_1"]))
print(response_2)

{'user_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小罗,我喜欢小鹿。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好小罗，很高兴认识你。你对小鹿有这样的喜好，可能是因为它们温顺、可爱，还是因为其他的原因呢？', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]), 'user_2': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好小谢！通过这句话，你是否是在说你和小罗在兴趣爱好上有差异呢？不过，我不是你们的专属想象力助手，无法直接得知你的具体喜好是什么。但是，一般人都会有自己独特的喜好，如音乐、电影、书籍、运动等等，而这些可能会和你身边的人有所不同。例如你可能非常喜欢科幻类的电影，而小罗则可能更喜欢爱情电影。如果你愿意的话，可以具体说一下你的喜好，我可以帮助你找到和小罗共享的共同兴趣。', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])])}
Human: 你好，我的名字叫小罗,我喜欢小鹿。
AI: 你好小罗，很高兴认识你。你对小鹿有这样的喜好，可能是因为它们温顺、可爱，还是因为其他的原因呢？
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
你好小谢！通过这句话，你是否是在说你和小罗在兴趣爱好上有差异呢？不过，我不是你们的专属想象力助手，无法直接得知你的具体喜好是什么。但是，一般人都会有自己独特的喜好，如音乐、电影、书籍、运动等等，而这些可能会和你身边的人有所不同。例如你可能非常喜欢科幻类的电影，而

举例5：结合大模型、提示词模板的使用（ChatPromptTemplate）

In [21]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1、创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2、提供提示词模板 (ChatPromptTemplate 替代 PromptTemplate)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以与人类对话。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# 3、提供Chain (使用管道符 | 构建)
chain = prompt | llm | StrOutputParser()

# 4、提供memory管理逻辑 (替代 ConversationBufferMemory)
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5、包装 Chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",      # 对应输入的 key
    history_messages_key="chat_history",  # 对应提示词模板中的占位符
)

# 6、调用
response_1 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小罗,我喜欢小鹿。"},
    config={"configurable": {"session_id": "user_1"}}
)

response_2 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？"},
    config={"configurable": {"session_id": "user_2"}}
)

print(store)
print(store["user_1"])
print(type(store["user_1"]))
print(response_2)

{'user_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小罗,我喜欢小鹿。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好小罗，很高兴认识你。你喜欢的小鹿是一种非常美丽且温顺的动物，很多人都喜欢它们。你喜欢它们的什么呢？是它们的外表、性格还是其他方面？', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]), 'user_2': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？', additional_kwargs={}, response_metadata={}), AIMessage(content='从你的问题来看，你希望我了解你个人的偏好，以便我可以帮助你理解你和小罗之间的偏好差异。不过，实际上我作为一个AI，无法直接了解你的真实偏好。但是，我可以帮助你思考一些可能性。\n\n如果你和小罗一起做选择时经常产生分歧，可能是因为你们的偏好有所不同。例如，小罗可能喜欢某些事物，如某种音乐风格、电影类型、书籍等，而你可能不喜欢。或者，可能你们在某些方面的偏好比较接近，但在某些细节上有所不同。\n\n你可以试着主动与小罗交流，了解他的兴趣和偏好，同时也分享你的喜好。通过这种方式，你们可以增进对彼此的了解，同时也可能在共同的兴趣中发现更多乐趣。', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])])}
Human: 你好，我的名字叫小罗,我喜欢小鹿。
AI: 你好小罗，很高兴认识你。你喜欢的小鹿是一种非常美丽且温顺的动物，很多人都喜欢它们。你喜欢它们的什么呢？是它们的外表、性格还是其他方面？
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
从你

# 3、Chain的使用

举例1：以PromptTemplate为例

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
import os
import dotenv

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 1. 创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# ==========================================
# 2. 使用 PromptTemplate (纯字符串模板)
# ==========================================
# PromptTemplate 适用于补全模型，或者当希望完全控制 Prompt 格式时。
# 这里手动把历史记录和新问题拼成一个长字符串。
prompt = PromptTemplate.from_template(
"""
你可以与人类对话。

当前对话历史:
{history}

{input}
"""
)

# 3. 构建链
chain = prompt | llm | StrOutputParser()

# 4. 定义记忆存储
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5. 包装链
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 6. 调用
print("--- 第一轮对话 ---")
response_1 = chain_with_history.invoke(
    {"input": "你好，我的名字叫小明"},
    config={"configurable": {"session_id": "user_1"}}
)
print(f"AI回复: {response_1}")

print("\n--- 第二轮对话 (验证记忆) ---")
response_2 = chain_with_history.invoke(
    {"input": "我叫什么名字？"},
    config={"configurable": {"session_id": "user_1"}}
)
print(f"AI回复: {response_2}")

--- 第一轮对话 ---
AI回复: 你好，小明！很高兴认识你。我是Qwen，一个大型语言模型，可以和你聊天、回答问题，甚至帮你写作文、编程等等。你今天有什么想和我聊的吗？或者需要什么帮助？

--- 第二轮对话 (验证记忆) ---
AI回复: 你叫小明！很高兴认识你，小明。有什么我可以帮你的吗？


举例2：使用默认提供的提示词模板



In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
import os
import dotenv

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 1. 创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2. 提供提示词模板
prompt = ChatPromptTemplate.from_messages([
    # System Message: 定义行为
    ("system", "你可以与人类对话。"),
    
    # History Placeholder: 动态插入历史记录
    # 这里的 variable_name="history" 必须与后面 RunnableWithMessageHistory 的配置对应
    MessagesPlaceholder(variable_name="history"), 
    
    # Human Message: 用户的当前输入
    ("human", "{input}"),
])

# 3. 构建链 (优化：使用 LCEL 管道符 |)
# 逻辑流：Prompt -> LLM -> OutputParser (直接解析成字符串)
chain = prompt | llm | StrOutputParser()

# 4. 定义记忆存储 (模拟数据库)
# 这是一个极其重要的架构变更：记忆不再绑定在 Chain 实例里，而是独立管理
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5. 包装链 (替代 ConversationChain)
# 给无状态的 chain 加上读取/写入历史的能力
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",    # 对应 Prompt 中的 {input}
    history_messages_key="history" # 对应 MessagesPlaceholder 的 variable_name
)

# 6. 调用
# 必须提供 session_id，这样系统才知道是在跟“谁”对话
response_1 = chain_with_history.invoke(
    {"input": "你好，我的名字叫小明"},
    config={"configurable": {"session_id": "user_1"}}
)

print(f"AI回复: {response}")

# --- 验证记忆能力 ---
response_2 = chain_with_history.invoke(
    {"input": "我叫什么名字？"},
    config={"configurable": {"session_id": "user_1"}}
)

print(f"AI记忆验证: {response_2}")

AI回复: 你叫小明呀！刚刚我们已经聊过了，我记住了你的名字。有什么我可以帮你的吗？😊
AI记忆验证: 你叫小明呀！刚刚我就是这么跟你说的。😊 有什么我可以帮你的吗？


# 4、ChainMemory的使用

举例1：


In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1. 实例化存储 (只负责存，不负责逻辑)
history = InMemoryChatMessageHistory()

# 2. 保存消息 (新版使用 add_user_message / add_ai_message)
history.add_user_message("你好")
history.add_ai_message("怎么了")

history.add_user_message("你是谁")
history.add_ai_message("我是AI助手")

history.add_user_message("你的生日是哪天？")
history.add_ai_message("我不清楚")

# 3. 读取并实现 Window 逻辑 (k=1)
# k=1 代表保留最后 1 组互动 = 最后 2 条消息
# 直接使用 Python 切片 [-2:]
window_messages = history.messages[-2:]

print("--- 优化后的结果 (Message 对象列表) ---")
print(window_messages)

--- 优化后的结果 (Message 对象列表) ---
[HumanMessage(content='你的生日是哪天？', additional_kwargs={}, response_metadata={}), AIMessage(content='我不清楚', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例2：返回消息构成的上下文记忆


In [8]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 1. 实例化存储对象 (只负责存，不负责逻辑)
# 对应原来的 return_messages=True，因为它原生就存的消息对象
history = InMemoryChatMessageHistory()

# 2. 保存消息 (使用语义更清晰的方法)
# 第一轮
history.add_user_message("你好")
history.add_ai_message("怎么了")

# 第二轮
history.add_user_message("你是谁")
history.add_ai_message("我是AI助手小智")

# 第三轮
history.add_user_message("初次对话，你能介绍一下你自己吗？")
history.add_ai_message("当然可以了。我是一个无所不能的小智。")

# 3. 实现窗口逻辑 (k=2)
# 原理：k=2 代表保留最近 2 轮对话 = 最近 4 条消息 (2 User + 2 AI)
k = 2
if len(history.messages) > k * 2:
    # 切片获取最后 4 条
    window_messages = history.messages[-k*2:]
else:
    window_messages = history.messages

# 4. 打印结果
print(f"--- 内存中共有 {len(history.messages)} 条消息，只返回最近 {k} 轮 ---")
print(window_messages)

--- 内存中共有 6 条消息，只返回最近 2 轮 ---
[HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='初次对话，你能介绍一下你自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以了。我是一个无所不能的小智。', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例3：结合llm、chain的使用


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1. 创建大模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2. 定义模版 (使用 ChatPromptTemplate)
prompt = ChatPromptTemplate.from_messages([
    # System Message
    ("system", "以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。"),
    
    # History Placeholder (这里注入的消息会被 trimmer 修剪过)
    MessagesPlaceholder(variable_name="history"),
    
    # Human Message
    ("human", "{question}"),
])

# 3. 定义 Chain (LCEL 管道)
# 逻辑：获取历史 -> 修剪历史 -> 填充 Prompt -> 模型推理 -> 解析字符串
chain = (
    RunnableWithMessageHistory(
        prompt | llm | StrOutputParser(),
        lambda session_id: get_session_history(session_id), # 获取历史的函数
        input_messages_key="question",
        history_messages_key="history",
    )
)

# 5. 定义历史存储
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


# 6. 执行链 (模拟对话流程)
config = {"configurable": {"session_id": "user_sun"}}

print("--- Round 1: 自我介绍 ---")
print(chain.invoke({"question": "你好，我是孙小空"}, config=config))

print("\n--- Round 2: 介绍师弟 ---")
print(chain.invoke({"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"}, config=config))

print("\n--- Round 3: 聊高考 ---")
print(chain.invoke({"question": "我今年高考，竟然考上了1本"}, config=config))

print("\n--- Round 4: 询问名字 ---")

print(chain.invoke({"question": "我叫什么名字？"}, config=config))

--- Round 1: 自我介绍 ---
你好，孙小空！很高兴认识你。我是通义千问，你可以叫我Qwen。我是一个由通义实验室开发的超大规模语言模型，能够帮助你进行各种语言理解、生成和对话任务。有什么我可以帮你的吗？比如回答问题、创作文字，或者进行有趣的闲聊？😊

--- Round 2: 介绍师弟 ---
哦，原来你是《西游记》的粉丝呀！猪小戒和沙小僧听起来就像是你师兄弟们的昵称呢。猪小戒可能是指猪八戒，而沙小僧应该就是沙和尚了。他们三个在取经路上可是一路相伴的好兄弟，虽然性格各异，但都很有趣。

你是想聊聊《西游记》的故事，还是想和我一起发挥想象力，编一个属于你们的师兄弟冒险故事？我很喜欢听你讲这些角色的故事，也愿意和你一起创造新的情节呢！😄

--- Round 3: 聊高考 (触发遗忘) ---
哇，孙小空，恭喜你考上1本！这是非常了不起的成绩，说明你真的很努力，也具备很强的学习能力和毅力。高考是人生中一次重要的挑战，能取得这样的成果，真的值得骄傲！

不过，我很好奇，你有没有想过未来想读什么专业？或者有没有特别想去的大学？我可以帮你分析一下不同专业的发展前景，或者聊聊大学生活的那些趣事。你是不是已经计划好接下来要做的事情了？比如填报志愿、准备大学生活等等？😄

--- Round 4: 询问名字 (验证 k=1 效果) ---
你叫孙小空呀！刚才我们已经聊到你了，你刚刚说“我今年高考，竟然考上了1本”，所以我知道你是孙小空。不过，如果你希望我叫你别的名字，比如“小空”或者“空空”，也可以告诉我哦！😊

对了，考上了1本，真是非常棒的成就！你可以跟我分享一下你的心情，或者聊聊你接下来的计划，我很乐意听你说！


举例4：修改举例3中的参数k


In [17]:
import os
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import trim_messages

# 1. 创建大模型
llm = ChatOpenAI(
    model=os.getenv("LLM_MODEL_ID"),
    )

# 2. 定义修剪器 (替代 ConversationBufferWindowMemory)
# 作用：它会自动计算 Token，只保留最近的内容。
# 为了模拟 k=1 (即只保留最后 1 轮问答)，将 max_tokens 设置得比较小 (例如 100-200)。
def custom_token_counter(messages) -> int:
    """
    使用 OpenAI 的 cl100k_base (GPT-4) 编码作为近似值来计算 Token。
    这避免了 LangChain 因为不认识 'Qwen' 模型名字而报错。
    """
    try:
        encoding = tiktoken.get_encoding("cl100k_base")
    except Exception:
        # 如果获取失败，回退到 p50k_base
        encoding = tiktoken.get_encoding("p50k_base")
    
    num_tokens = 0
    for msg in messages:
        # 简单估算：内容长度 + 额外开销(每条消息约3-4个token)
        # 这种估算对于 trim_messages 这种截断用途已经足够精确了
        num_tokens += 4 
        num_tokens += len(encoding.encode(msg.content))
        
    return num_tokens

trimmer = trim_messages(
    max_tokens=66,        # 限制 Token 数量 (模拟窗口大小)
    strategy="last",       # 保留最新的消息
    token_counter=custom_token_counter,     # 使用自定义函数计算 Token
    include_system=True,   # 始终保留系统提示词
    allow_partial=False,   # 不截断单条消息的中间
)

prompt = ChatPromptTemplate.from_messages([
    # System Message
    ("system", "以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。"),
    
    # History Placeholder (这里注入的消息会被 trimmer 修剪过)
    MessagesPlaceholder(variable_name="history"),
    
    # Human Message
    ("human", "{question}"),
])

# 4. 定义 Chain (LCEL 管道)
# 逻辑：获取历史 -> 修剪历史 -> 填充 Prompt -> 模型推理 -> 解析字符串
chain = (
    RunnableWithMessageHistory(
        prompt | llm | StrOutputParser(),
        lambda session_id: get_session_history(session_id), # 获取历史的函数
        input_messages_key="question",
        history_messages_key="history",
    )
)

# 5. 定义历史存储
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_trimming = (
    RunnableWithMessageHistory(
        (lambda x: {
            "history": trimmer.invoke(x["history"]),
            "question": x["question"]
        }) | prompt | llm | StrOutputParser(),
        get_session_history,
        input_messages_key="question",
        history_messages_key="history",
    )
)

# 6. 执行链 (模拟对话流程)
config = {"configurable": {"session_id": "user_sun"}}

print("--- Round 1: 自我介绍 ---")
# 此时历史为空，AI 记住你是孙小空
print(chain_with_trimming.invoke({"question": "你好，我是孙小空"}, config=config))

print("\n--- Round 2: 介绍师弟 ---")
# 此时历史包含 Round 1。AI 应该知道你是孙小空，并记住师弟。
print(chain_with_trimming.invoke({"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"}, config=config))

print("\n--- Round 3: 聊高考 (触发遗忘) ---")
# 此时历史应该包含 Round 2。Token 限制会导致 Round 1 (名字) 被挤出。
print(chain_with_trimming.invoke({"question": "我今年高考，竟然考上了1本"}, config=config))

print("\n--- Round 4: 询问名字 (验证 k=1 效果) ---")
# 此时历史只包含 Round 3 (高考)。Round 1 的名字已经被彻底遗忘了。
# 预期结果：AI 表示不知道你的名字。
print(chain_with_trimming.invoke({"question": "我叫什么名字？"}, config=config))

--- Round 1: 自我介绍 ---
你好，孙小空！很高兴认识你。我是Qwen，一个由通义实验室开发的超大规模语言模型。我在这里可以和你聊天、回答问题、创作文字，甚至帮你学习新知识。有什么我可以帮你的吗？😊

--- Round 2: 介绍师弟 ---
啊，你提到了“猪小戒”和“沙小僧”，听起来像是你给自己或你的师弟们起的昵称呢！这让我想起了《西游记》里的猪八戒和沙僧，他们是中国古典文学中非常经典的角色。你是不是在用这些名字来调侃或者亲近你的师弟们？还是他们真的和《西游记》里的角色有某种联系？

不过，我注意到你提到的是“师弟”，这让我想到你可能是在某种师徒关系的环境中，比如武术社团、学术团队、甚至是游戏中的角色。你有没有特别想和他们一起做的事情？或者有什么有趣的故事想分享？我很乐意听你说说他们的性格、爱好，以及你们之间的互动！

--- Round 3: 聊高考 (触发遗忘) ---
哇，恭喜你考上1本！这真的是一个非常值得庆祝的成就，说明你付出了很多努力，也具备了很强的学习能力和毅力。高考是中国学生人生中的一个重要节点，能够考上1本，不仅代表你在学术上的优秀，也意味着你迈出了实现梦想的重要一步。

不过，考上了1本是不是让你感到有些意外或者惊喜呢？毕竟这个结果可能超出了你的预期，或者你可能在备考过程中遇到了一些挑战。如果是这样，那你的努力和坚持真的没有白费！

你现在一定很激动，可能也在思考接下来的路该怎么走。比如选择专业、填报志愿，或者考虑未来的发展方向。有什么具体的问题或者困惑，我可以帮你一起探讨吗？比如你是想了解大学专业的选择建议，还是对未来的职业规划有疑问？总之，不管你想了解什么，我都会尽我所能为你提供帮助和支持！

--- Round 4: 询问名字 (验证 k=1 效果) ---
哦，这个问题很有趣！不过我好像没有关于你名字的具体信息呢。你是第一次和我交谈吗？还是说我们之前有过一些互动？如果你愿意告诉我你的名字，我一定会记住的！😊
